# Advanced Data Science with IBM
## Use Case: Fault prediction for gear bearings (anomaly detection)

### Model training

In [1]:
#All import statements
import numpy as np
import json
import pandas as pd
from pandas import DataFrame
import matplotlib.pyplot as plt
import seaborn as sns
from keras.models import Sequential
from keras import regularizers
from keras.models import load_model

from botocore.client import Config
import ibm_boto3
%matplotlib inline

Using TensorFlow backend.


In [2]:
# import of csv files
def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share your notebook.
client_b2d04bb960e541b59bf748286f30fef3 = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='HXB6bIyaokBGw-eWQ7IlD3GvZyZhVYDCePt5uiSYMZnq',
    ibm_auth_endpoint="https://iam.bluemix.net/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3-api.us-geo.objectstorage.service.networklayer.com')

body = client_b2d04bb960e541b59bf748286f30fef3.get_object(Bucket='courseraadvanceddatascience-donotdelete-pr-eqpoqlvyxje0ta',Key='df_nok.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_nok = pd.read_csv(body)
df_nok.index = pd.DatetimeIndex(df_nok.date)
df_nok = df_nok.drop(['date'], axis=1)

In [3]:
body = client_b2d04bb960e541b59bf748286f30fef3.get_object(Bucket='courseraadvanceddatascience-donotdelete-pr-eqpoqlvyxje0ta',Key='df_ok.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_ok = pd.read_csv(body)
df_ok.index = pd.DatetimeIndex(df_ok.date)
df_ok = df_ok.drop(['date'], axis=1)

#### 1. Loading models

In [4]:
# load LSTM model
lstm_model = load_model('lstm_model.h5')

# load Dense model
dense_model = load_model('dense_model.h5')

Instructions for updating:
Colocations handled automatically by placer.


#### 2. Training LSTM model

In [5]:
def train_lstm(data, model, iterations):
    # data: reshaped data array
    # model: lstm model
    #iteration: int with number of training iterations
    
    for i in range(iterations):
        model_lstm = model.fit(data, data, epochs=100, batch_size=72, validation_data=(data, data), verbose=1, shuffle=False)
    return model_lstm

In [6]:
#reshape the dataframe according to the LSTM input shape
def reshape_lstm(df,timesteps,dim=4):
    #df: dataframe
    #timesteps: int with number of timesteps
    #dim: int with number ot timesteps
    
    arr = np.array(df)
    samples=len(arr)
    trim = samples %  timesteps
    reshaped = arr[:samples-trim]
    reshaped.shape = (int((samples-trim)/timesteps),timesteps,dim)
    return reshaped

In [7]:
# reshape the data for LSTM model
lstm_ok = reshape_lstm(df_ok,10,dim=4)
lstm_nok = reshape_lstm(df_nok,10,dim=4)

In [8]:
#details about the number of parameters and output shapes of each layer and an overall model structure
lstm_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_7 (LSTM)                (None, 10, 50)            11000     
_________________________________________________________________
lstm_8 (LSTM)                (None, 10, 50)            20200     
_________________________________________________________________
lstm_9 (LSTM)                (None, 10, 50)            20200     
_________________________________________________________________
dense_16 (Dense)             (None, 10, 4)             204       
Total params: 51,604
Trainable params: 51,604
Non-trainable params: 0
_________________________________________________________________


In [9]:
# train the model
lstm_hist = train_lstm(lstm_ok,lstm_model,3)

Instructions for updating:
Use tf.cast instead.
Train on 22 samples, validate on 22 samples
Epoch 1/100
22/22 [==============================] - 6s 289ms/step - loss: 0.3689 - val_loss: 0.3616
Epoch 2/100
22/22 [==============================] - 0s 12ms/step - loss: 0.3616 - val_loss: 0.3515
Epoch 3/100
22/22 [==============================] - 0s 12ms/step - loss: 0.3515 - val_loss: 0.3408
Epoch 4/100
22/22 [==============================] - 0s 12ms/step - loss: 0.3408 - val_loss: 0.3291
Epoch 5/100
22/22 [==============================] - 0s 12ms/step - loss: 0.3291 - val_loss: 0.3161
Epoch 6/100
22/22 [==============================] - 0s 13ms/step - loss: 0.3161 - val_loss: 0.3013
Epoch 7/100
22/22 [==============================] - 0s 13ms/step - loss: 0.3013 - val_loss: 0.2844
Epoch 8/100
22/22 [==============================] - 0s 12ms/step - loss: 0.2844 - val_loss: 0.2649
Epoch 9/100
22/22 [==============================] - 0s 12ms/step - loss: 0.2649 - val_loss: 0.2420
Epoch 1

22/22 [==============================] - 0s 13ms/step - loss: 0.0502 - val_loss: 0.0495
Epoch 80/100
22/22 [==============================] - 0s 12ms/step - loss: 0.0495 - val_loss: 0.0488
Epoch 81/100
22/22 [==============================] - 0s 13ms/step - loss: 0.0488 - val_loss: 0.0482
Epoch 82/100
22/22 [==============================] - 0s 13ms/step - loss: 0.0482 - val_loss: 0.0476
Epoch 83/100
22/22 [==============================] - 0s 12ms/step - loss: 0.0476 - val_loss: 0.0469
Epoch 84/100
22/22 [==============================] - 0s 12ms/step - loss: 0.0469 - val_loss: 0.0463
Epoch 85/100
22/22 [==============================] - 0s 11ms/step - loss: 0.0463 - val_loss: 0.0457
Epoch 86/100
22/22 [==============================] - 0s 14ms/step - loss: 0.0457 - val_loss: 0.0451
Epoch 87/100
22/22 [==============================] - 0s 12ms/step - loss: 0.0451 - val_loss: 0.0444
Epoch 88/100
22/22 [==============================] - 0s 11ms/step - loss: 0.0444 - val_loss: 0.0439
Epo

Epoch 60/100
22/22 [==============================] - 0s 13ms/step - loss: 0.0184 - val_loss: 0.0182
Epoch 61/100
22/22 [==============================] - 0s 13ms/step - loss: 0.0182 - val_loss: 0.0181
Epoch 62/100
22/22 [==============================] - 0s 12ms/step - loss: 0.0181 - val_loss: 0.0179
Epoch 63/100
22/22 [==============================] - 0s 15ms/step - loss: 0.0179 - val_loss: 0.0177
Epoch 64/100
22/22 [==============================] - 0s 14ms/step - loss: 0.0177 - val_loss: 0.0175
Epoch 65/100
22/22 [==============================] - 0s 12ms/step - loss: 0.0175 - val_loss: 0.0173
Epoch 66/100
22/22 [==============================] - 0s 11ms/step - loss: 0.0173 - val_loss: 0.0171
Epoch 67/100
22/22 [==============================] - 0s 12ms/step - loss: 0.0171 - val_loss: 0.0169
Epoch 68/100
22/22 [==============================] - 0s 12ms/step - loss: 0.0169 - val_loss: 0.0167
Epoch 69/100
22/22 [==============================] - 0s 15ms/step - loss: 0.0167 - val_los

Epoch 41/100
22/22 [==============================] - 0s 12ms/step - loss: 0.0082 - val_loss: 0.0081
Epoch 42/100
22/22 [==============================] - 0s 14ms/step - loss: 0.0081 - val_loss: 0.0080
Epoch 43/100
22/22 [==============================] - 0s 11ms/step - loss: 0.0080 - val_loss: 0.0079
Epoch 44/100
22/22 [==============================] - 0s 12ms/step - loss: 0.0079 - val_loss: 0.0078
Epoch 45/100
22/22 [==============================] - 0s 13ms/step - loss: 0.0078 - val_loss: 0.0077
Epoch 46/100
22/22 [==============================] - 0s 13ms/step - loss: 0.0077 - val_loss: 0.0075
Epoch 47/100
22/22 [==============================] - 0s 12ms/step - loss: 0.0075 - val_loss: 0.0075
Epoch 48/100
22/22 [==============================] - 0s 13ms/step - loss: 0.0075 - val_loss: 0.0074
Epoch 49/100
22/22 [==============================] - 0s 14ms/step - loss: 0.0074 - val_loss: 0.0072
Epoch 50/100
22/22 [==============================] - 0s 13ms/step - loss: 0.0072 - val_los

#### 3. Training Dense model

In [10]:
# Modeling - train neural network
def train_NN(data, model,batch_size=10,epochs=100):
    # data: dataframe
    
    history=model.fit(np.array(data),np.array(data),
                      batch_size=batch_size, 
                      epochs=epochs,
                      validation_split=0.05,
                      verbose = 1)
    return history

In [11]:
#details about the number of parameters and output shapes of each layer and an overall model structure
dense_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_17 (Dense)             (None, 10)                50        
_________________________________________________________________
dense_18 (Dense)             (None, 2)                 22        
_________________________________________________________________
dense_19 (Dense)             (None, 10)                30        
_________________________________________________________________
dense_20 (Dense)             (None, 4)                 44        
Total params: 146
Trainable params: 146
Non-trainable params: 0
_________________________________________________________________


In [12]:
dense_hist = train_NN(df_ok, dense_model)

Train on 212 samples, validate on 12 samples
Epoch 1/100
212/212 [==============================] - 1s 6ms/step - loss: 0.1347 - val_loss: 0.1016
Epoch 2/100
212/212 [==============================] - 0s 237us/step - loss: 0.0752 - val_loss: 0.0406
Epoch 3/100
212/212 [==============================] - 0s 199us/step - loss: 0.0214 - val_loss: 0.0053
Epoch 4/100
212/212 [==============================] - 0s 205us/step - loss: 0.0017 - val_loss: 2.0088e-04
Epoch 5/100
212/212 [==============================] - 0s 240us/step - loss: 9.1144e-05 - val_loss: 6.6600e-05
Epoch 6/100
212/212 [==============================] - 0s 227us/step - loss: 5.4520e-05 - val_loss: 4.7717e-05
Epoch 7/100
212/212 [==============================] - 0s 224us/step - loss: 3.3790e-05 - val_loss: 4.3117e-05
Epoch 8/100
212/212 [==============================] - 0s 202us/step - loss: 3.3142e-05 - val_loss: 4.0536e-05
Epoch 9/100
212/212 [==============================] - 0s 201us/step - loss: 3.2868e-05 - val_los

212/212 [==============================] - 0s 237us/step - loss: 3.0838e-05 - val_loss: 3.9796e-05
Epoch 75/100
212/212 [==============================] - 0s 175us/step - loss: 3.3084e-05 - val_loss: 5.5738e-05
Epoch 76/100
212/212 [==============================] - 0s 196us/step - loss: 3.2000e-05 - val_loss: 4.0564e-05
Epoch 77/100
212/212 [==============================] - 0s 200us/step - loss: 3.0586e-05 - val_loss: 4.1269e-05
Epoch 78/100
212/212 [==============================] - 0s 247us/step - loss: 3.0672e-05 - val_loss: 3.2242e-05
Epoch 79/100
212/212 [==============================] - 0s 199us/step - loss: 3.1214e-05 - val_loss: 4.8184e-05
Epoch 80/100
212/212 [==============================] - 0s 197us/step - loss: 3.0770e-05 - val_loss: 4.4613e-05
Epoch 81/100
212/212 [==============================] - 0s 222us/step - loss: 3.1254e-05 - val_loss: 4.9960e-05
Epoch 82/100
212/212 [==============================] - 0s 177us/step - loss: 3.1663e-05 - val_loss: 4.3313e-05
Epoch

#### 4. Training KNN model

In [13]:
# creation, training and evaluation of the knn model will be done in the evaluation notebook

### Saving of the keras history object

In [14]:
# Get the dictionary containing each metric and the loss for each epoch
history_lstm = lstm_hist.history
history_dense = dense_hist.history

# Save it under the form of a json file
json.dump(history_lstm, open('lstm_hist', 'w'))
json.dump(history_dense, open('dense_hist', 'w'))
print('saved')

saved
